In [9]:
import pandas as pd
from random import randint

ds = pd.read_csv('TRNcod.xls', delimiter = "\t")

In [ ]:
# print([d for d in ds.columns])

In [11]:
# inadimplentes = pd.DataFrame(list(filter(lambda x: x == 1, ds['IND_BOM_1_2'])))

# Selecionando quem é inadimplente
inadimplentes = ds[ds['IND_BOM_1_2'] == 1]

# Selecionando quem é adimplente
adimplente = ds[ds['IND_BOM_1_2'] == 0]

# Dividindo datasets
global treino_ina
global teste_ina
global valid_ina
global treino_adi
global teste_adi
global valid_adi

# Equiparando tamanho de inadimplentes ao de adimplentes
# def repeticao():
treino_ina = inadimplentes[:int(len(inadimplentes)/2)]
teste_ina  = inadimplentes[int(len(inadimplentes)/2):int((len(inadimplentes)*3)/4)]
valid_ina  = inadimplentes[int((len(inadimplentes)*3)/4):]

treino_adi = adimplente[:int(len(adimplente)/2)]
teste_adi  = adimplente[int(len(adimplente)/2):int((len(adimplente)*3)/4)]
valid_adi  = adimplente[int((len(adimplente)*3)/4):]

while(len(treino_ina) < len(treino_adi)):
    nova_linha = treino_ina.iloc[randint(0, len(treino_ina))]
    # print(len(treino_ina), '|', len(treino_adi), ' ', end="")
    treino_ina = pd.concat([treino_ina, nova_linha], ignore_index=True)

while(len(valid_ina) < len(valid_adi)):
    nova_linha = teste_ina.iloc[randint(0, len(teste_ina))]
    # print(len(teste_ina), '|', len(teste_adi), ' ', end="")
    valid_ina  = pd.concat([valid_ina, nova_linha], ignore_index=True)

# repeticao()

print('Tamanhos de inadimplentes: \nTreino: {}\nTeste: {}\nValidação: {}\n'.format(len(treino_ina), len(teste_ina), len(valid_ina)))
print('Tamanhos de adimplentes: \nTreino: {}\nTeste: {}\nValidação: {}'.format(len(treino_adi), len(teste_adi), len(valid_adi)))

Tamanhos de inadimplentes: 
Treino: 127563
Teste: 33524
Validação: 63783

Tamanhos de adimplentes: 
Treino: 127549
Teste: 63774
Validação: 63775


In [12]:
print('Tamanhos de inadimplentes: \nTreino: {}\nTeste: {}\nValidação: {}\n'.format(len(treino_ina), len(teste_ina), len(valid_ina)))

Tamanhos de inadimplentes: 
Treino: 127563
Teste: 33524
Validação: 63783

